In [3]:
import lxml.etree as ET
from spacytei.tei import TeiReader

In [4]:
certainty_col, _ = SkosCollection.objects.get_or_create(name='certainty')

In [5]:
file = "http://127.0.0.1:8080/exist/restxq/rita/api/collections/indices/listbibl.xml"

In [6]:
doc = TeiReader(file)
ns_tei = doc.ns_tei
ns_tei

{'tei': 'http://www.tei-c.org/ns/1.0'}

In [7]:
for x in doc.any_xpath('//tei:bibl'):
    xml_id = x.xpath('./@xml:id', namespaces=ns_tei)[0]
    temp_work, _ = Work.objects.get_or_create(legacy_id=xml_id)
    try:
        title = x.xpath('./tei:title/text()', namespaces=ns_tei)[0]
    except IndexError:
        title = 'kein Titel'
    try:
        cer = x.xpath('./tei:title/@cer', namespaces=ns_tei)[0]
    except IndexError:
        cer = 'keine Angaben'
    cer_obj, _ = SkosConcept.objects.get_or_create(pref_label=cer)
    cer_obj.collection.add(certainty_col)
    temp_work.title = title
    temp_work.title_certainty = cer_obj
    creator_objects = []
    creators = x.xpath('./tei:*[name()="author" or name()="publisher"]', namespaces=ns_tei)
    if len(creators) == 0:
        pass
    else:
        for creator in creators:
            if creator.text is not None:
                temp_creator, _ = Creator.objects.get_or_create(
                    name=creator.text
                )
                try:
                    auth_cer = creator.xpath('./@cer')[0]
                except IndexError:
                    auth_cer = None
                if auth_cer is not None:
                    cer_obj, _ = SkosConcept.objects.get_or_create(pref_label=auth_cer)
                    cer_obj.collection.add(certainty_col)
                    temp_creator.creator_certainty = cer_obj
                try:
                    norm_id = creator.xpath('./@ref')[0]
                except IndexError:
                    norm_id = None
                if norm_id is not None:
                    temp_creator.normdata_id = norm_id
                temp_creator.save()
                creator_objects.append(temp_creator)
    temp_work.creator.set(creator_objects)
    temp_work.save()
    exemp = x.xpath('./tei:exemplar', namespaces=ns_tei)
    if len(exemp) > 0:
        for ex in exemp:
            if ex.text:
                link, _ = Exemplar.objects.get_or_create(normdata_id=ex.text)
                try:
                    cer = ex.attrib['cer']
                except KeyError:
                    cer = 'keine Angaben'
                cer_obj, _ = SkosConcept.objects.get_or_create(pref_label=cer)
                cer_obj.collection.add(certainty_col)
                link.certainty = cer_obj
                link.related_work = temp_work
                link.save()
                 

In [ ]:
# WorkExample.objects.all().delete()

In [2]:
Exemplar.objects.all().delete()

(2458, {'books.Exemplar': 1229, 'idprovider.IdProvider': 1229})